In [31]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from enum import Enum
import re



In [32]:
# Set whether to display selium browser
VISIBLE = True

options = Options()
# If not visible do not display browser to user
if not VISIBLE: options.add_argument('--headless')

url = "https://goblin.bet/#/"
driver = webdriver.Firefox(options=options)
driver.get(url)




In [33]:
contButt = WebDriverWait(driver, timeout=30).until(lambda d: d.find_element(By.CLASS_NAME, "WelcomeStart.Left"))
contButt.click()

In [34]:
pageSrc = driver.page_source
soup = BeautifulSoup(pageSrc)

RIGHT = True
LEFT = False

def getLog(soup: BeautifulSoup, printLog = True):
    # Find log
    log = soup.find("div", {"class": "scrollhost BLogScroll"})
    # If print log true print out the log
    if printLog:
        for lg in log.find_all("div", {"class": "LogText"}):
            print(lg.text)
    # return the betting log
    return log

def getMoney(soup: BeautifulSoup):
    return soup.find('span', {"class": "BetsScore"}).text

def getCreatureSoup(soup: BeautifulSoup, side=LEFT):
    if side == LEFT:
        return soup.find("div", {"class": "Block Statsheet Left TeamRed"})
    return soup.find("div", {"class": "Block Statsheet Right TeamBlue"})
        

+ Ghast takes 6 Fire damage!
+ The Grick's Tentacles hits!
+ Grick attacks Ghast with a Tentacles.
+ Grick saves against Paralyzed.
+ Grick takes 4 Slashing damage!
+ The Ghast's Claws hits!
+ Ghast attacks Grick with a Claws.
+ Grick resists Paralyzed (CON) : SUCCESS
+ Grick is incapacitated.
+ Grick saves against Poisoned.
+ Ghast emits an awful stench.
+ Ghast is no longer Hidden (Attacking).
+ Grick takes 4 Slashing damage!
+ Grick becomes Paralyzed (8 vs DC:10).
+ Grick takes 6 Slashing damage!
+ The Ghast's Claws hits!
+ Ghast attacks Grick with a Claws.
+ Ghast runs 25 feet forwards.
+ Ghast Wins Initiative!
+ Ghast Rolls Initiative: 18
+ Grick Rolls Initiative: 15
+ ---Round Start!---
+ Round begins in 5 seconds!
+ Round begins in 10 seconds!
+ Round begins in 20 seconds!
+ Round begins in 30 seconds!
+ Challenger 2: Ghast
+ Challenger 1: Grick
+ ------NEW BATTLE------ +
+ Wizard wins! Giant Elk is eliminated!
+ Wizard drains 11 Max HP from Giant Elk.
+ Wizard heals for 11 hp!


In [96]:
statsList = soup.find("div", {"class": "SSStats"})
print(statsList.prettify())

<div class="SSStats">
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  ATTRIBUTES
 </span>
 <span class="Stat">
  STR: 14
 </span>
 <span class="Stat">
  DEX: 14
 </span>
 <span class="Stat">
  CON: 11
 </span>
 <span class="Stat">
  INT: 3
 </span>
 <span class="Stat">
  WIS: 14
 </span>
 <span class="Stat">
  CHA: 5
 </span>
 <span class="Stat Highlight">
  HP: 12/26
 </span>
 <span class="Stat Highlight">
  AC: 14
 </span>
 <span class="Stat Highlight">
  Spd: 30ft
 </span>
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  RESIST
 </span>
 <span class="Stat Mod">
  <span>
   Bludgeoning, Piercing, Slashing, Fire
   <span class="PiercedResist">
   </span>
  </span>
 </span>
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  VULNERABLE
 </span>
 <span class="Stat Mod">
  <span>
   Cold
   <span class="PiercedResist">
   </span>
  </span>
 </span>
 <span class="SSHeader">
 

In [111]:
# Strips all alphabetic characters
def stripChrs(word):
    return re.sub("[^0-9]", '', word)

# Strips all nonalphanumeric characters in string
def stripInts(word):
    return re.sub(r'\W+', '', word)

# Finds a tag with a specfic class name
def findSpanClass(soup, className, ALL=False):
    if not ALL:
        return soup.find("span", {"class": className})
    soup.find_all("span", {"class": className})



class Creature:

    def __init__(self, soup: BeautifulSoup, side=LEFT):
        self.side = side
        self.soup = getCreatureSoup(soup, side)
        self.name = self.InitName(self.soup)
        self.cr = self.InitCR(self.soup)
        self.InitStats()
        
    def InitName(self, soup: BeautifulSoup):
        return findSpanClass(soup, "SSName").text
    
    def InitCR(self, soup: BeautifulSoup):
        return re.sub("[^0-9]", '', findSpanClass(soup, "SSInfo").text)

    def getAction(self, action: BeautifulSoup):
        action = [findSpanClass(action, "ActName").text, findSpanClass(action, "ActDesc").text]
        return action

    # Initializes all stats in the creature sheet
    def InitStats(self):
        # Initialize stat categories
        self.stats = None
        self.immunities = None
        self.resists = None
        self.actions = None
        self.conditions = None

        # Gets list of all tags in the stats soup
        statsList =  self.soup.find("div", {"class": "SSStats"})
        # Go through every header value in the tags
        for head in statsList.find_all("span", {"class": "SSHeader"}):
            # Record the text of the header
            headTxt = stripInts(head.text)
            if headTxt == "ATTRIBUTES":
                self.stats = self.getAttributes(statsList)
            # If content is immunities
            if headTxt == "IMMUNE":
                # Store immunities using next tag
                self.immunities = [re.split(',', word) for word in head.next.text]
            if headTxt == "RESIST":
                # Store resistances using next tag
                self.resists = [re.split(',', word) for word in head.next.text]
            if headTxt == "ACTIONS":
                self.actions = []
                # Append each action to list
                self.InitActions(head)
            if headTxt == "CONDITIONS":
                # Find all conditions
                self.conditions = [condition.text for condition in statsList.find_all("div", {"class": "Stat Small"})]
            
    # Initializes possible actions of the 
    def InitActions(self, actionHead):
        child = actionHead.findNext('div')
        print(actionHead)
        print(child)
        try:
            while child.name == 'div':
                
                print(child.text)
                for action in child.find_all("div", {"class": "Stat Wide"}):
                    print(action)
                    self.actions.append(self.getAction(action))
                child = child.next

        except:
            print(child)
            None

    
    # Copies attributes tag
    def getAttributes(self, statsList: BeautifulSoup):
        stats = {"STR": None, "DEX": None, "CON": None, "INT": None, "WIS": None, "CHA": None}
        for i, stat in enumerate(statsList.find_all("span", {"class": "Stat"})):
            if i == 0:
                stats["STR"] = stripChrs(stat.text)
            elif i == 1:
                stats["DEX"] = stripChrs(stat.text)
            elif i == 2:
                stats["CON"] = stripChrs(stat.text)
            elif i == 3:
                stats["INT"] = stripChrs(stat.text)
            elif i == 4:
                stats["WIS"] = stripChrs(stat.text)
            elif i == 5:
                stats["CHA"] = stripChrs(stat.text)
            elif i == 6:
                hpVals = [val for val in re.split("/", stat.text)]
                stats["HP"] = stripChrs(hpVals[0])
                stats["HPMax"] = stripChrs(hpVals[1])
            elif i == 7:
                stats["AC"] = stripChrs(stat.text)
            elif i == 8:
                stats["SPD"] = stripChrs(stat.text)
        return stats
        

        # Assign Stats
        #stats["STR"] = statsList[0] #re.sub("[^0-9]", '', statsList[0].text)

soup = BeautifulSoup(driver.page_source)
cret = Creature(soup, RIGHT)


<span class="SSHeader"><img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/> ACTIONS</span>
<div class="Stat Wide"><span class="ActName">Bite: </span><span class="ActDesc">+5, 2d6+3 Slashing, 1d6 Acid, Adhered (If Large)</span></div>
Bite: +5, 2d6+3 Slashing, 1d6 Acid, Adhered (If Large)


In [105]:
cret.name
cret.stats
cret.actions

[]

In [99]:
r = requests.get("https://goblin.bet/#/").text
soup = BeautifulSoup(r)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="Monsters fight for your pleasure, throw gold coins at them" name="description"/>
  <title>
   Goblin Bet
  </title>
  <link href="/static/css/main.bc1c7931.chunk.css" rel="stylesheet"/>
 </head>
 <body>
  <noscript>
   You need to enable JavaScript to run this app.
  </noscript>
  <div id="root">
  </div>
  <script>
   !function(e){function t(t){for(var n,l,i=t[0],f=t[1],a=t[2],c=0,s=[];c<i.length;c++)l=i[c],Object.prototype.hasOwnProperty.call(o,l)&&o[l]&&s.push(o[l][0]),o[l]=0;for(n in f)Object.prototype.hasOwnProperty.call(f,n)&&(e[n]=f[n]);for(p&&p(t);s.length;)s.shift()();return u.push.apply(u,a||[]),r()}function r(){for(var e,t=0;t<u.length;t++){for(var r=u[t],n=!0,i=1;i<r.length;i++){var f=r[i];0!==o[f]&&(n=!1)}n&&(u.splice(t--,1),e=l(l.s=r[0]))}return e}var n={},o={1:0},u=[];func